In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Softmax, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.utils import to_categorical
# from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import matplotlib.pyplot as plt
import pandas as pd
#from data_pp_mmwave import *
import os

data_dir = "/content/drive/My Drive/mmWaveData/"

step = 12
data  = read_from_pickle(data_dir+'data_sitting.pkl', step)
label = np.zeros(data.shape[0])

data_standing = read_from_pickle(data_dir+'data_standing.pkl', step)
data = np.vstack((data, data_standing))
label = np.append(label,np.ones(data_standing.shape[0]))

data_walking = read_from_pickle(data_dir+'data_walking.pkl', step)
data = np.vstack((data, data_walking))
label = np.append(label, 2 * np.ones(data_walking.shape[0]))
label = to_categorical(label)

data_final =  df.append({'A': i}, ignore_index=True)

X_train, X_test, y_train, y_test = train_test_split(data,label, test_size=0.2)

scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = StandardScaler()
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i]) 

for i in range(X_test.shape[2]):
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i]) 

opt  = Adam(lr=0.0001)
call_back = EarlyStopping(patience=2)
input_shape = X_train.shape[1:3]
model = Sequential()
model.add(LSTM(20, input_shape=input_shape, return_sequences=False))
model.add(Dense(10, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()
hist = model.fit(X_train, y_train, epochs=2000,
                validation_split=0.15,
                callbacks= [call_back],
                batch_size=32, verbose=2)

y_pred = model.predict(X_test)
results = model.evaluate(X_test,y_test,batch_size = 32)
print('test loss, test accuracy:',results)


In [ ]:
plt.figure(figsize=(10,8))
plt.plot(hist.history['loss'], label = 'Training Loss')
plt.plot(hist.history['val_loss'], label = 'Validation Loss')
plt.legend(loc = 'best')